Conditional GAN

In [ ]:
# Import necessary libraries
import numpy as np
import os
from glob import glob
from skimage.io import imread
from skimage import transform
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from google.colab import files
from zipfile import ZipFile

In [ ]:
from zipfile import ZipFile

# Specify the path to the zip files
train_zip_path = '/content/train_sample_edge2shoes.zip'
val_zip_path = '/content/val_edge2shoes.zip'

# Extract the training dataset
with ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./train_e2s/')

# Extract the validation dataset
with ZipFile(val_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./val_e2s/')

In [ ]:
# Print files to verify extraction
print("Training files: ", os.listdir('./train_e2s/'))
print("Validation files: ", os.listdir('./val_e2s/'))

Training files:  ['3596_AB.jpg', '3358_AB.jpg', '745_AB.jpg', '54_AB.jpg', '459_AB.jpg', '732_AB.jpg', '3877_AB.jpg', '749_AB.jpg', '446_AB.jpg', '3602_AB.jpg', '3_AB.jpg', '1194_AB.jpg', '51_AB.jpg', '3509_AB.jpg', '1191_AB.jpg', '3434_AB.jpg', '3370_AB.jpg', '1136_AB.jpg', '47_AB.jpg', '363_AB.jpg', '799_AB.jpg', '3622_AB.jpg', '3609_AB.jpg', '86_AB.jpg', '482_AB.jpg', '1169_AB.jpg', '3809_AB.jpg', '447_AB.jpg', '207_AB.jpg', '792_AB.jpg', '392_AB.jpg', '4355_AB.jpg', '1030_AB.jpg', '772_AB.jpg', '753_AB.jpg', '255_AB.jpg', '715_AB.jpg', '1099_AB.jpg', '761_AB.jpg', '4396_AB.jpg', '3366_AB.jpg', '497_AB.jpg', '164_AB.jpg', '104_AB.jpg', '3633_AB.jpg', '3414_AB.jpg', '3430_AB.jpg', '3374_AB.jpg', '831_AB.jpg', '375_AB.jpg', '1021_AB.jpg', '217_AB.jpg', '1176_AB.jpg', '222_AB.jpg', '829_AB.jpg', '3932_AB.jpg', '3769_AB.jpg', '3620_AB.jpg', '36_AB.jpg', '3355_AB.jpg', '3433_AB.jpg', '3369_AB.jpg', '90_AB.jpg', '3931_AB.jpg', '777_AB.jpg', '1426_AB.jpg', '1040_AB.jpg', '860_AB.jpg', '883

In [ ]:
def load_data(dataset_path, batch_size=1, is_val=False):
    # Load image paths from the dataset directory
    image_paths = glob(os.path.join(dataset_path, '*.jpg'))

    if len(image_paths) == 0:
        print(f"No images found in the path: '{dataset_path}'")
        return [], []

    # Randomly select a batch of images
    batch_images = np.random.choice(image_paths, size=batch_size, replace=False)
    img_res = (128, 128)  # Resolution to resize images
    imgs_A = []
    imgs_B = []

    # Display the list of images being loaded
    print(f"Images selected for loading (Total: {len(batch_images)}):")
    for img_path in batch_images:
        print(os.path.basename(img_path))  # Show image file names only

    for img_path in batch_images:
        # Load the image
        img = imread(img_path)

        h, w, _ = img.shape
        _w = int(w / 2)

        # Split the image into two halves A and B
        img_A, img_B = img[:, _w:, :], img[:, :_w, :]

        # Resize both halves to the required resolution
        img_A = transform.resize(img_A, img_res)
        img_B = transform.resize(img_B, img_res)

        # Optionally flip images for data augmentation if it's not validation
        if not is_val and np.random.random() < 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)

        # Append the processed images to the lists
        imgs_A.append(img_A)
        imgs_B.append(img_B)

    # Convert image lists to numpy arrays
    imgs_A = np.array(imgs_A)
    imgs_B = np.array(imgs_B)

    # Check if the images have the expected shape
    if imgs_A.shape[1:] != (128, 128, 3):
        raise ValueError(f"Unexpected shape of images: {imgs_A.shape[1:]}")

    # Normalize images to the range [-1, 1]
    imgs_A = imgs_A / 127.5 - 1.
    imgs_B = imgs_B / 127.5 - 1.

    print(f"\nLoaded {len(imgs_A)} image pairs with shapes: {imgs_A.shape}, {imgs_B.shape}")

    return imgs_A, imgs_B

# Step 2: Load the training and validation datasets
train_dataset = '/content/train_e2s'  # Path to training dataset
val_dataset = '/content/val_e2s'      # Path to validation dataset

# Load a specific number of images (or the whole dataset) for both train and val
batch_size_train = 128  # Specify the number of training images you want to load
batch_size_val = 32     # Specify the number of validation images you want to load

# Load training data
train_imgs_A, train_imgs_B = load_data(train_dataset, batch_size=batch_size_train, is_val=False)
print(f'\nTraining images loaded: {len(train_imgs_A)} pairs')

# Load validation data
val_imgs_A, val_imgs_B = load_data(val_dataset, batch_size=batch_size_val, is_val=True)
print(f'\nValidation images loaded: {len(val_imgs_A)} pairs')


Images selected for loading (Total: 128):
737_AB.jpg
3605_AB.jpg
430_AB.jpg
3594_AB.jpg
3613_AB.jpg
3604_AB.jpg
1159_AB.jpg
1439_AB.jpg
1031_AB.jpg
3584_AB.jpg
3411_AB.jpg
87_AB.jpg
837_AB.jpg
3791_AB.jpg
164_AB.jpg
759_AB.jpg
1119_AB.jpg
383_AB.jpg
758_AB.jpg
385_AB.jpg
3378_AB.jpg
358_AB.jpg
1146_AB.jpg
240_AB.jpg
1161_AB.jpg
263_AB.jpg
824_AB.jpg
1291_AB.jpg
670_AB.jpg
404_AB.jpg
378_AB.jpg
79_AB.jpg
1115_AB.jpg
1160_AB.jpg
439_AB.jpg
3624_AB.jpg
3782_AB.jpg
410_AB.jpg
541_AB.jpg
199_AB.jpg
1108_AB.jpg
3590_AB.jpg
3622_AB.jpg
789_AB.jpg
3522_AB.jpg
3628_AB.jpg
3769_AB.jpg
855_AB.jpg
291_AB.jpg
795_AB.jpg
1175_AB.jpg
511_AB.jpg
741_AB.jpg
1132_AB.jpg
1038_AB.jpg
858_AB.jpg
809_AB.jpg
4395_AB.jpg
452_AB.jpg
861_AB.jpg
667_AB.jpg
800_AB.jpg
4333_AB.jpg
1563_AB.jpg
143_AB.jpg
697_AB.jpg
3984_AB.jpg
786_AB.jpg
3427_AB.jpg
101_AB.jpg
3804_AB.jpg
3619_AB.jpg
655_AB.jpg
363_AB.jpg
3374_AB.jpg
63_AB.jpg
773_AB.jpg
486_AB.jpg
3618_AB.jpg
746_AB.jpg
1564_AB.jpg
1025_AB.jpg
484_AB.jpg
3359_AB.j

In [ ]:
# Define input shape based on loaded data
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)
gf=64
df=64

# Generator function
def build_generator():
    def conv2d(layer_input, filters, f_size=4, bn=True):
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(negative_slope=0.2)(d)
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = BatchNormalization(momentum=0.8)(u)
        u = Concatenate()([u, skip_input])
        return u

    d0 = Input(shape=img_shape)  # Ensure input shape matches loaded data

    # Downsampling layers
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)
    d5 = conv2d(d4, gf*8)
    d6 = conv2d(d5, gf*8)
    d7 = conv2d(d6, gf*8)

    # Upsampling layers
    u1 = deconv2d(d7, d6, gf*8)
    u2 = deconv2d(u1, d5, gf*8)
    u3 = deconv2d(u2, d4, gf*8)
    u4 = deconv2d(u3, d3, gf*4)
    u5 = deconv2d(u4, d2, gf*2)
    u6 = deconv2d(u5, d1, gf)

    u7 = UpSampling2D(size=2)(u6)
    output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

    return Model(d0, output_img)

# Discriminator function
def build_discriminator():
    def d_layer(layer_input, filters, f_size=4, bn=True):
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(negative_slope=0.2)(d)
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        return d

    img_A = Input(shape=img_shape)  # Real image input
    img_B = Input(shape=img_shape)  # Fake image input
    combined_imgs = Concatenate(axis=-1)([img_A, img_B])  # Combine real and fake images

    # Discriminator layers
    d1 = d_layer(combined_imgs, df, bn=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same', activation='sigmoid')(d4)  # Validity output

    return Model([img_A, img_B], validity)



# Calculate disc_patch after building the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='mse', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Define the disc_patch based on the discriminator's output shape
discriminator_output = discriminator.output_shape
print("Discriminator output shape:", discriminator_output)
disc_patch = discriminator.output_shape[1:]  # This will be (8, 8, 1)
print("Discriminator patch shape:", disc_patch)


Discriminator output shape: (None, 8, 8, 1)
Discriminator patch shape: (8, 8, 1)


In [ ]:
# Adjust the optimizer for both generator and discriminator
generator_optimizer = Adam(0.00005, 0.5)  # Lower learning rate for the generator
discriminator_optimizer = Adam(0.0001, 0.5)  # Original learning rate for the discriminator

# Compile Models with adjusted learning rates
def compile_models():
    # Build and compile the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='mse', optimizer=discriminator_optimizer, metrics=['accuracy'])

    # Build the generator
    generator = build_generator()

    # Define inputs for the combined model
    img_A = Input(shape=img_shape)
    img_B = Input(shape=img_shape)

    # Generate fake images
    fake_A = generator(img_B)

    # Discriminator's decision
    discriminator.trainable = True
    valid = discriminator([fake_A, img_B])

    # Combined model (for training the generator)
    combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
    combined.compile(loss=['mse', 'mae'], loss_weights=[1, 100], optimizer=generator_optimizer)

    return generator, discriminator, combined

# Update the training loop to train the generator more frequently
def train_models(dataset_path, epochs, batch_size, sample_interval=1):
    generator, discriminator, combined = compile_models()

    for epoch in range(epochs):
        num_batches = len(glob(os.path.join(dataset_path, '*.jpg'))) // batch_size

        for batch_i in range(num_batches):
            imgs_A, imgs_B = load_data(dataset_path, batch_size=batch_size, is_val=False)

            if len(imgs_A) == 0 or len(imgs_B) == 0:
                print(f"Skipping empty batch at batch index {batch_i}.")
                continue  # Skip empty batches

            # Create labels for the discriminator
            valid = np.ones((batch_size,) + disc_patch)
            fake = np.zeros((batch_size,) + disc_patch)

            # Generate fake images
            fake_A = generator.predict(imgs_B)

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch([imgs_A, imgs_B], valid)
            d_loss_fake = discriminator.train_on_batch([fake_A, imgs_B], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator more frequently
            if batch_i % 2 == 0:  # Train generator every 2 batches
                g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                # Print progress
                print(f"Epoch {epoch}/{epochs} - Batch {batch_i}/{num_batches} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss[0]:.4f}]")
            else:
                print(f"Epoch {epoch}/{epochs} - Batch {batch_i}/{num_batches} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%]")

        # Validation and visualization every sample_interval epochs
        if epoch % sample_interval == 0:
            validate(dataset_path, generator, epoch)
            fake_A = generator.predict(imgs_B)
            for i in range(len(fake_A)):
                plt.imsave(f'generated_image_epoch_{epoch}_batch_{i}.jpg', (fake_A[i] + 1) / 2)

    print("Training complete.")


In [ ]:
# generator=build_generator()
# generator.summary()

In [ ]:
# # Print the discriminator's architecture and output shape
# discriminator.summary()


In [ ]:

# Fixing the validate function (normalize images for display)
def validate(val_dataset_path, generator, epoch, examples=5, dim=(2, 5), figsize=(15, 6)):
    imgs_A, imgs_B = load_data(val_dataset_path, batch_size=examples, is_val=True)
    fake_A = generator.predict(imgs_B)

    # Rescale images from [-1, 1] to [0, 1] for display
    gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A], axis=0)
    gen_imgs = (gen_imgs + 1) / 2  # Rescale to [0, 1]

    titles = ['Input', 'Generated', 'Ground Truth']
    fig, axs = plt.subplots(dim[0], dim[1], figsize=figsize)
    cnt = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            if cnt < len(gen_imgs):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[cnt % 3])
                axs[i, j].axis('off')
                cnt += 1
            else:
                axs[i, j].axis('off')
    plt.tight_layout()
    plt.savefig(f'validation_images_epoch_{epoch}.jpg')
    plt.show()

# Ensure the generator and validation dataset path are correctly set
generator, _, _ = compile_models()  # Get the trained generator

# Train the model
train_models(train_dataset, epochs=5, batch_size=32, sample_interval=1)

# Validate the results and visualize after training
validate(val_dataset, generator, epoch=10)


NameError: name 'compile_models' is not defined

In [ ]:
# from tensorflow.keras.utils import plot_model

# def create_cgan_architecture_diagram():
#     # Ensure models are built
#     generator, discriminator, combined = compile_models()

#     # Save and plot the combined model
#     plot_model(combined, to_file='cgan_combined_model.png', show_shapes=True, show_layer_names=True)

#     print("cGAN combined model diagram saved as 'cgan_combined_model.png'.")

# create_cgan_architecture_diagram()


In [ ]:
# import matplotlib.pyplot as plt
# import cv2

# # Load a specific image (replace with your image path)
# image_path = '/content/train_e2s/224_AB.jpg'
# image = cv2.imread(image_path)

# # Convert the image from BGR (OpenCV default) to RGB
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Display the image
# plt.imshow(image)
# plt.axis('off')  # Hide axes
# plt.show()


In [ ]:
def evaluate_model(generator, val_dataset_path, num_samples=100):
    imgs_A, imgs_B = load_data(val_dataset_path, batch_size=num_samples, is_val=True)
    fake_A = generator.predict(imgs_B)

    # Calculate the Mean Absolute Error (MAE) or any other metric as required
    mae = np.mean(np.abs(fake_A - imgs_A))
    print(f"Mean Absolute Error (MAE) on validation set: {mae:.4f}")

# Call the evaluate_model function after training
evaluate_model(generator, val_dataset, num_samples=100)


Cycle GAN

In [1]:
import zipfile
import os

# Define your zip file paths
day_zip_path = '/content/day.zip'  # Replace with your actual path
night_zip_path = '/content/night.zip'  # Replace with your actual path

# Define directories to extract images
os.makedirs('day_images', exist_ok=True)
os.makedirs('night_images', exist_ok=True)

# Unzip the day images
with zipfile.ZipFile(day_zip_path, 'r') as zip_ref:
    zip_ref.extractall('day_images')

# Unzip the night images
with zipfile.ZipFile(night_zip_path, 'r') as zip_ref:
    zip_ref.extractall('night_images')


In [2]:
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

# Function to load and resize images
def load_and_resize_images(image_folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust based on your image formats
            img = Image.open(os.path.join(image_folder, filename))
            img = img.resize(target_size)
            images.append(np.array(img))
    return np.array(images)

# Load images
day_images = load_and_resize_images('day_images')
night_images = load_and_resize_images('night_images')

# Split datasets into train and test sets
train_day, test_day = train_test_split(day_images, test_size=0.2, random_state=42)
train_night, test_night = train_test_split(night_images, test_size=0.2, random_state=42)

print(f"Training day images: {len(train_day)}, Test day images: {len(test_day)}")
print(f"Training night images: {len(train_night)}, Test night images: {len(test_night)}")


Training day images: 417, Test day images: 105
Training night images: 181, Test night images: 46


In [3]:
import tensorflow as tf

def create_dataset(day_images, night_images, batch_size=1):
    # Create TensorFlow datasets
    train_day_dataset = tf.data.Dataset.from_tensor_slices(day_images)
    train_night_dataset = tf.data.Dataset.from_tensor_slices(night_images)

    # Combine datasets into a single dataset
    train_dataset = tf.data.Dataset.zip((train_day_dataset, train_night_dataset))

    # Shuffle, repeat, and batch the dataset
    train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset

batch_size = 1  # Change based on your model requirements
train_dataset = create_dataset(train_day, train_night, batch_size)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers

def build_generator():
    inputs = layers.Input(shape=(256, 256, 3))

    # Downsampling
    x = layers.Conv2D(64, kernel_size=7, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    # Residual blocks
    skip_connections = []
    for _ in range(9):
        skip = x
        x = layers.Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        x = layers.Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
        x = layers.add([x, skip])  # Skip connection

    # Upsampling
    x = layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.ReLU()(x)
    outputs = layers.Conv2D(3, kernel_size=7, strides=1, padding='same', activation='tanh')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

def build_discriminator():
    model = tf.keras.Sequential()

    model.add(layers.Input(shape=(256, 256, 3)))  # Input shape (256, 256, 3)
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(512, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(1, kernel_size=4, strides=1, padding='same'))  # Output for discriminator

    return model

# Create instances of your generator and discriminator
generator_g = build_generator()  # Day to Night
generator_f = build_generator()  # Night to Day
discriminator_x = build_discriminator()  # Night Discriminator
discriminator_y = build_discriminator()  # Day Discriminator

# Print model summaries
generator_g.summary()
generator_f.summary()
discriminator_x.summary()
discriminator_y.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 256, 256, 64)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 128, 128, 128)  │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 64, 64, 256)    │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ leaky_re_lu_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU) │ (None, 64, 64, 256)    │              0 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ leaky_re_lu_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 64, 64, 256)    │              0 │ conv2d_4[0][0],        │
│                           │                        │                │ leaky_re_lu_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU) │ (None, 64, 64, 256)    │              0 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ leaky_re_lu_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 64, 64, 256)    │              0 │ conv2d_6[0][0],        │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_5 (LeakyReLU) │ (None, 64, 64, 256)    │              0 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ leaky_re_lu_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)          

 Total params: 11,378,179 (43.40 MB)

 Trainable params: 11,378,179 (43.40 MB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_49 (Conv2D)                   │ (None, 128, 128, 64)        │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_28 (LeakyReLU)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_50 (Conv2D)                   │ (None, 64, 64, 128)         │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_29 (LeakyReLU)           │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_51 (Conv2D)                   │ (None, 32, 32, 256)         │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_30 (LeakyReLU)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_52 (Conv2D)                   │ (None, 16, 16, 512)         │       2,097,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_31 (LeakyReLU)           │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_53 (Conv2D)                   │ (None, 16, 16, 1)           │           8,193 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,764,737 (10.55 MB)

 Trainable params: 2,764,737 (10.55 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
import tensorflow as tf

# Loss functions
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(generated):
    """Calculate generator loss."""
    return loss_object(tf.ones_like(generated), generated)

def discriminator_loss(real, generated):
    """Calculate discriminator loss."""
    real_loss = loss_object(tf.ones_like(real), real)
    fake_loss = loss_object(tf.zeros_like(generated), generated)
    return real_loss + fake_loss

def cycle_consistency_loss(real_image, cycled_image, lambda_cycle=10):
    # Convert images to float32 to ensure compatibility
    real_image = tf.cast(real_image, tf.float32)
    cycled_image = tf.cast(cycled_image, tf.float32)

    # L1 loss between the original image and the cycled image
    loss = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return lambda_cycle * loss  # Weighted by lambda_cycle (usually 10)



def identity_loss(real_image, same_image, lambda_identity=0.5):
    """Calculate identity loss."""
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return lambda_identity * loss

# Optimizers
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [14]:
@tf.function
def train_step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # Generate images
        fake_y = generator_g(real_x, training=True)  # Day to Night
        cycled_x = generator_f(fake_y, training=True)  # Night back to Day

        fake_x = generator_f(real_y, training=True)  # Night to Day
        cycled_y = generator_g(fake_x, training=True)  # Day back to Night

        # Discriminator predictions
        disc_real_x = discriminator_x(real_x, training=True)
        disc_fake_x = discriminator_x(fake_x, training=True)

        disc_real_y = discriminator_y(real_y, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # Losses
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        # Add cycle consistency loss
        cycle_loss = cycle_consistency_loss(real_x, cycled_x) + cycle_consistency_loss(real_y, cycled_y)
        total_gen_g_loss = gen_g_loss + cycle_loss
        total_gen_f_loss = gen_f_loss + cycle_loss

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

    # Apply gradients
    gradients_g = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
    gradients_f = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)
    gradients_disc_x = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
    gradients_disc_y = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)

    generator_g_optimizer.apply_gradients(zip(gradients_g, generator_g.trainable_variables))
    generator_f_optimizer.apply_gradients(zip(gradients_f, generator_f.trainable_variables))
    discriminator_x_optimizer.apply_gradients(zip(gradients_disc_x, discriminator_x.trainable_variables))
    discriminator_y_optimizer.apply_gradients(zip(gradients_disc_y, discriminator_y.trainable_variables))

    return total_gen_g_loss, total_gen_f_loss, disc_x_loss, disc_y_loss


In [ ]:
# Train your model
EPOCHS = 10  # Adjust the number of epochs as needed

for epoch in range(EPOCHS):
    total_gen_g_loss = 0
    total_gen_f_loss = 0
    total_disc_x_loss = 0
    total_disc_y_loss = 0

    for real_x, real_y in train_dataset:
        gen_g_loss, gen_f_loss, disc_x_loss, disc_y_loss = train_step(real_x, real_y)

        # Accumulate the losses over the epoch
        total_gen_g_loss += gen_g_loss
        total_gen_f_loss += gen_f_loss
        total_disc_x_loss += disc_x_loss
        total_disc_y_loss += disc_y_loss

    # Print out the losses for the epoch
    print(f'Epoch {epoch + 1}/{EPOCHS} completed')
    print(f'  Generator G loss: {total_gen_g_loss:.4f}, Generator F loss: {total_gen_f_loss:.4f}')
    print(f'  Discriminator X loss: {total_disc_x_loss:.4f}, Discriminator Y loss: {total_disc_y_loss:.4f}')


In [ ]:
generator_g.save('generator_g.h5')
generator_f.save('generator_f.h5')
discriminator_x.save('discriminator_x.h5')
discriminator_y.save('discriminator_y.h5')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Function to visualize images
def visualize_transformation(original_image, transformed_image, title):
    plt.figure(figsize=(8, 4))

    # Plot original image
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow((original_image + 1) / 2)  # Rescale image to [0, 1] for viewing
    plt.axis('off')

    # Plot transformed image
    plt.subplot(1, 2, 2)
    plt.title(title)
    plt.imshow((transformed_image + 1) / 2)  # Rescale image to [0, 1] for viewing
    plt.axis('off')

    plt.show()

# Load and preprocess image for prediction (example for a day-to-night transformation)
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(256, 256))  # Reshape if needed
    img = img_to_array(img)
    img = (img / 127.5) - 1  # Normalize to [-1, 1] range
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Visualize a day image transformed to night
day_image_path = '/content/day_images/106.jpg'
original_day_image = preprocess_image(day_image_path)

# Predict the transformation (Day to Night)
predicted_night_image = generator_g.predict(original_day_image)

# Remove the batch dimension for visualization
original_day_image = np.squeeze(original_day_image)
predicted_night_image = np.squeeze(predicted_night_image)

# Visualize the original and transformed image
visualize_transformation(original_day_image, predicted_night_image, title='Day to Night')
